In [1]:
!pip install numpy

In [2]:
import numpy as np

In [3]:
!pip install pandas

In [4]:
!pip install opencv-python

   ---------------------------------------- 0.0/38.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/38.6 MB 1.4 MB/s eta 0:00:29
   ---------------------------------------- 0.2/38.6 MB 2.5 MB/s eta 0:00:16
    --------------------------------------- 0.7/38.6 MB 6.1 MB/s eta 0:00:07
   - -------------------------------------- 1.5/38.6 MB 10.5 MB/s eta 0:00:04
   - -------------------------------------- 1.8/38.6 MB 9.3 MB/s eta 0:00:04
   -- ------------------------------------- 2.2/38.6 MB 9.3 MB/s eta 0:00:04
   -- ------------------------------------- 2.2/38.6 MB 8.9 MB/s eta 0:00:05
   -- ------------------------------------- 2.8/38.6 MB 8.9 MB/s eta 0:00:05
   -- ------------------------------------- 2.9/38.6 MB 8.3 MB/s eta 0:00:05
   --- ------------------------------------ 3.4/38.6 MB 8.0 MB/s eta 0:00:05
   --- ------------------------------------ 3.4/38.6 MB 8.1 MB/s eta 0:00:05
   --- ------------------------------------ 3.7/38.6 MB 7.7 MB/s eta 0:00:05
   --

In [7]:
!pip install moviepy

     ---------------------------------------- 0.0/388.3 kB ? eta -:--:--
     --- ----------------------------------- 30.7/388.3 kB 1.4 MB/s eta 0:00:01
     ----------- -------------------------- 112.6/388.3 kB 1.7 MB/s eta 0:00:01
     -------------------------------------  378.9/388.3 kB 4.0 MB/s eta 0:00:01
     -------------------------------------  378.9/388.3 kB 4.0 MB/s eta 0:00:01
     -------------------------------------  378.9/388.3 kB 4.0 MB/s eta 0:00:01
     -------------------------------------  378.9/388.3 kB 4.0 MB/s eta 0:00:01
     -------------------------------------- 388.3/388.3 kB 1.5 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
   ---------------------------------------- 0.0/22.6 MB ? eta -:--:--
    --------------------------------------- 0.6/22.6 MB 17.0 MB/s eta 0:00:02
   -- ------------------------------------- 1.5/22.6 MB 23.7 MB/s eta 0:00:01
   -- ---------------------------------

In [8]:
import matplotlib.pyplot as plt
import numpy as np
import cv2
import os
import matplotlib.image as mpimg
from moviepy.editor import VideoFileClip
import math

In [9]:
def interested_region(img, vertices):
    if len(img.shape) > 2: 
        mask_color_ignore = (255,) * img.shape[2]
    else:
        mask_color_ignore = 255
        
    cv2.fillPoly(np.zeros_like(img), vertices, mask_color_ignore)
    return cv2.bitwise_and(img, np.zeros_like(img))

In [10]:
def hough_lines(img, rho, theta, threshold, min_line_len, max_line_gap):
    lines = cv2.HoughLinesP(img, rho, theta, threshold, np.array([]), minLineLength=min_line_len, maxLineGap=max_line_gap)
    line_img = np.zeros((img.shape[0], img.shape[1], 3), dtype=np.uint8)
    lines_drawn(line_img,lines)
    return line_img

In [15]:
def lines_drawn(img, lines, color=[255, 0, 0], thickness=6):
    global cache
    global first_frame

    slope_l, slope_r = [], []
    lane_l, lane_r = [], []
    α = 0.2

    for line in lines:
        for x1, y1, x2, y2 in line:
            slope = (y2 - y1) / (x2 - x1)
            if slope > 0.4:
                slope_r.append(slope)
                lane_r.append(line)
            elif slope < -0.4:
                slope_l.append(slope)
                lane_l.append(line)

    img.shape[0] = min(y1, y2, img.shape[0])

    if ((len(lane_l) == 0) or (len(lane_r) == 0)):
        print('no lane detected')
        return 1

    slope_mean_l = np.mean(slope_l, axis=0)
    slope_mean_r = np.mean(slope_r, axis=0)
    mean_l = np.mean(np.array(lane_l), axis=0)
    mean_r = np.mean(np.array(lane_r), axis=0)

    if ((slope_mean_r == 0) or (slope_mean_l == 0)):
        print('dividing by zero')
        return 1

        x1_l = int((img.shape[0] - mean_l[0][1] - (slope_mean_l * mean_l[0][0])) / slope_mean_l)
        x2_l = int((img.shape[0] - mean_l[0][1] - (slope_mean_l * mean_l[0][0])) / slope_mean_l)
        x1_r = int((img.shape[0] - mean_r[0][1] - (slope_mean_r * mean_r[0][0])) / slope_mean_r)
        x2_r = int((img.shape[0] - mean_r[0][1] - (slope_mean_r * mean_r[0][0])) / slope_mean_r)

    if x1_l > x1_r:
        x1_l = int((x1_l + x1_r) / 2)
        x1_r = x1_l

    
        y1_l = int((slope_mean_l * x1_l) + mean_l[0][1] - (slope_mean_l * mean_l[0][0]))
        y1_r = int((slope_mean_r * x1_r) + mean_r[0][1] - (slope_mean_r * mean_r[0][0]))
        y2_l = int((slope_mean_l * x2_l) + mean_l[0][1] - (slope_mean_l * mean_l[0][0]))
        y2_r = int((slope_mean_r * x2_r) + mean_r[0][1] - (slope_mean_r * mean_r[0][0]))

    else:
        y1_l = img.shape[0]
        y2_l = img.shape[0]
        y1_r = img.shape[0]
        y2_r = img.shape[0]

    present_frame = np.array([x1_l, y1_l, x2_l, y2_l, x1_r, y1_r, x2_r, y2_r], dtype="float32")

    if first_frame == 1:
        next_frame = present_frame
        first_frame = 0
    
    else:
        prev_frame = cache
        next_frame = (1 - α) * prev_frame + α * present_frame

    cv2.line(img, (int(next_frame[0]), int(next_frame[1])), (int(next_frame[2]), int(next_frame[3])), color, thickness)
    cv2.line(img, (int(next_frame[4]), int(next_frame[5])), (int(next_frame[6]), int(next_frame[7])), color, thickness)

    cache = next_frame

In [16]:
def weighted_img(img, initial_img, α=0.8, β=1., λ=0.):
    return cv2.addWeighted(initial_img, α, img, β, λ)


def process_image(image):

    global first_frame

    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    img_hsv = cv2.cvtColor(image, cv2.COLOR_RGB2HSV)


    lower_yellow = np.array([20, 100, 100], dtype = "uint8")
    upper_yellow = np.array([30, 255, 255], dtype="uint8")

    mask_yellow = cv2.inRange(img_hsv, lower_yellow, upper_yellow)
    mask_white = cv2.inRange(gray_image, 200, 255)
    mask_yw = cv2.bitwise_or(mask_white, mask_yellow)
    mask_yw_image = cv2.bitwise_and(gray_image, mask_yw)

    gauss_gray= cv2.GaussianBlur(mask_yw_image, (5, 5), 0)

    canny_edges=cv2.Canny(gauss_gray, 50, 150)

    imshape = image.shape
    lower_left = [imshape[1]/9,imshape[0]]
    lower_right = [imshape[1]-imshape[1]/9,imshape[0]]
    top_left = [imshape[1]/2-imshape[1]/8,imshape[0]/2+imshape[0]/10]
    top_right = [imshape[1]/2+imshape[1]/8,imshape[0]/2+imshape[0]/10]
    vertices = [np.array([lower_left,top_left,top_right,lower_right],dtype=np.int32)]
    roi_image = interested_region(canny_edges, vertices)

    theta = np.pi/180

    line_image = hough_lines(roi_image, 4, theta, 30, 100, 180)
    result = weighted_img(line_image, image, α=0.8, β=1., λ=0.)
    return result

In [23]:
first_frame = 1
white_output = r"C:\Users\HP\OneDrive\.ipynb_checkpoints\video.mp4"
clip1 = VideoFileClip(r"C:\Users\HP\OneDrive\.ipynb_checkpoints\video.mp4")
white_clip = clip1.fl_image(process_image)
white_clip.write_videofile(white_output, audio=False)




                                                               


t:  12%|█▏        | 15/125 [10:39<00:01, 73.78it/s, now=None]e]


                                                               


t:  12%|█▏        | 15/125 [10:39<00:01, 73.78it/s, now=None]e]

Moviepy - Building video C:\Users\HP\OneDrive\.ipynb_checkpoints\video.mp4.
Moviepy - Writing video C:\Users\HP\OneDrive\.ipynb_checkpoints\video.mp4




t:  99%|█████████▉| 359/363 [00:03<00:00, 93.75it/s, now=None] 
                                                              


                                                               


t:  12%|█▏        | 15/125 [10:42<00:01, 73.78it/s, now=None]e]


                                                               


t:  12%|█▏        | 15/125 [10:42<00:01, 73.78it/s, now=None]e]

Moviepy - Done !
Moviepy - video ready C:\Users\HP\OneDrive\.ipynb_checkpoints\video.mp4


In [1]:
import matplotlib.pyplot as plt
import numpy as np
import cv2
import os
import matplotlib.image as mpimg
from moviepy.editor import VideoFileClip
import math
# Initialize cache and first_frame variables
cache = None
first_frame = True
# Function to mask the region of interest in the image
def interested_region(img, vertices):
    mask = np.zeros_like(img)
    if len(img.shape) > 2:
        mask_color_ignore = (255,) * img.shape[2]
    else:
        mask_color_ignore = 255
        
    cv2.fillPoly(mask, vertices, mask_color_ignore)
    return cv2.bitwise_and(img, mask)
# Function to apply Hough Transform and draw lane lines
def hough_lines(img, rho, theta, threshold, min_line_len, max_line_gap):
    lines = cv2.HoughLinesP(img, rho, theta, threshold, np.array([]), minLineLength=min_line_len, maxLineGap=max_line_gap)
    line_img = np.zeros((img.shape[0], img.shape[1], 3), dtype=np.uint8)
    lines_drawn(line_img, lines)
    return line_img
# Function to draw the detected lines on the image
def lines_drawn(img, lines, color=[255, 0, 0], thickness=6):
    global cache
    global first_frame

    slope_l, slope_r = [], []
    lane_l, lane_r = [], []
    α = 0.2
# Classify lines based on their slopes
    for line in lines:
        for x1, y1, x2, y2 in line:
            slope = (y2 - y1) / (x2 - x1)
            if slope > 0.4:
                slope_r.append(slope)
                lane_r.append(line)
            elif slope < -0.4:
                slope_l.append(slope)
                lane_l.append(line)

    if ((len(lane_l) == 0) or (len(lane_r) == 0)):
        print('no lane detected')
        return
# Calculate mean slopes and intercepts for left and right lanes
    slope_mean_l = np.mean(slope_l, axis=0)
    slope_mean_r = np.mean(slope_r, axis=0)
    mean_l = np.mean(np.array(lane_l), axis=0)
    mean_r = np.mean(np.array(lane_r), axis=0)

    if ((slope_mean_r == 0) or (slope_mean_l == 0)):
        print('dividing by zero')
        return
# Define y-coordinates for lane lines
    y1 = img.shape[0]
    y2 = int(img.shape[0] * 0.6)
# Calculate x-coordinates for left and right lanes
    x1_l = int((y1 - mean_l[0][1]) / slope_mean_l + mean_l[0][0])
    x2_l = int((y2 - mean_l[0][1]) / slope_mean_l + mean_l[0][0])
    x1_r = int((y1 - mean_r[0][1]) / slope_mean_r + mean_r[0][0])
    x2_r = int((y2 - mean_r[0][1]) / slope_mean_r + mean_r[0][0])
# Store current frame's coordinates
    present_frame = np.array([x1_l, y1, x2_l, y2, x1_r, y1, x2_r, y2], dtype="float32")
# Smooth the lane lines using exponential moving average
    if first_frame:
        next_frame = present_frame
        first_frame = False
    else:
        prev_frame = cache
        next_frame = (1 - α) * prev_frame + α * present_frame
# Draw the smoothed lane lines on the image
    cv2.line(img, (int(next_frame[0]), int(next_frame[1])), (int(next_frame[2]), int(next_frame[3])), color, thickness)
    cv2.line(img, (int(next_frame[4]), int(next_frame[5])), (int(next_frame[6]), int(next_frame[7])), color, thickness)
# Update cache with current frame
    cache = next_frame
# Function to process each frame of the video
def process_image(image):
    # Convert image to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    # Apply Gaussian blur
    blur = cv2.GaussianBlur(gray, (5, 5), 0)
    # Detect edges using Canny edge detector
    edges = cv2.Canny(blur, 50, 150)
    # Define region of interest
    imshape = image.shape
    vertices = np.array([[(0, imshape[0]), (450, 320), (490, 320), (imshape[1], imshape[0])]], dtype=np.int32)
    masked_edges = interested_region(edges, vertices)
    # Hough Transform parameters 
    rho = 2
    theta = np.pi/180
    threshold = 15
    min_line_len = 40
    max_line_gap = 20
    # Detect and draw lines
    line_image = hough_lines(masked_edges, rho, theta, threshold, min_line_len, max_line_gap)
    # Overlay the line image on the original image
    result = cv2.addWeighted(image, 0.8, line_image, 1, 0)
    return result
# Define the input and output video paths
white_output = r"C:\Users\HP\OneDrive\.ipynb_checkpoints\image.mp4" 
clip1 = VideoFileClip(r"C:\Users\HP\OneDrive\.ipynb_checkpoints\image.mp4")
# Process the video and save the output
white_clip = clip1.fl_image(process_image)
white_clip.write_videofile(white_output, audio=False)

Moviepy - Building video C:\Users\HP\OneDrive\.ipynb_checkpoints\image.mp4.
Moviepy - Writing video C:\Users\HP\OneDrive\.ipynb_checkpoints\image.mp4



Moviepy - Done !
Moviepy - video ready C:\Users\HP\OneDrive\.ipynb_checkpoints\image.mp4
